In [ ]:
# Optionally install dependencies
# !pip install -r ../requirements.txt
# !pip freeze >> ../../requirements.txt

# Imports and constants
from utils.secrets import DATA_PATH
from typing import Any

import sys
import os
import warnings

import re

import requests
import zipfile
import io

from datetime import datetime

from time import sleep
from tqdm.notebook import tqdm

import numpy as np
import polars as pl

from chembl_webresource_client.new_client import new_client as chembl_client

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Scaffolds import MurckoScaffold

sys.path.append(os.path.abspath("../../"))


warnings.filterwarnings("ignore", category=UserWarning)

os.environ['VERBOSE'] = 'True'
API_DELAY_SECONDS = 0.3

FDA_DATA_PATH = os.path.join(DATA_PATH, 'drugsfda_20251127')

In [21]:
# Auxiliary Functions
def vprint(*args: Any, **kwargs: Any) -> None:
    """
    Conditionally prints output only if the 'VERBOSE' environmental
    variable is set to a truthy value (e.g., 'True', '1', 'YES').

    It accepts all the standard arguments of the built-in print() function.

    Args:
        *args: Positional arguments to be printed.
        **kwargs: Keyword arguments to be passed to the print() function.
    """
    verbose_value = os.environ.get("VERBOSE", "False").lower()

    if verbose_value in ("true", "1", "t", "y", "yes"):
        print(*args, **kwargs)

In [22]:
# # Download and extract FDA Drugs@FDA data
# # https://www.fda.gov/drugs/drug-approvals-and-databases/drugsfda-data-files

# FDA_DRUGS_URL = "https://www.fda.gov/media/89850/download"

# def download_fda_drugs_data(target_path: str) -> str:
#     """Downloads and extracts the FDA Drugs@FDA dataset.
    
#     Args:
#         target_path: Directory where the data should be extracted.
        
#     Returns:
#         The name of the extracted folder (e.g., 'datdaf20241013').
#     """
#     headers = {
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     }
#     response = requests.get(FDA_DRUGS_URL, headers=headers)
#     response.raise_for_status()
    
#     with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
#         folder_name = "drugsfda_" + datetime.now().strftime("%Y%m%d")
#         zf.extractall(target_path + "/" + folder_name)
    
#     print(f"Extracted FDA data to: {os.path.join(target_path, folder_name)}")
#     return folder_name

# fda_folder_name = download_fda_drugs_data(DATA_PATH)
# FDA_DATA_PATH = os.path.join(DATA_PATH, fda_folder_name)

In [23]:
# # Review all available files
# for file in os.listdir(os.path.join(FDA_DATA_PATH)):
#     print(f"Reading file: {file}")
#     try:
#         df = pl.read_csv(
#             os.path.join(FDA_DATA_PATH, file),
#             separator="\t",
#             encoding="latin-1",
#             ignore_errors=True,
#             null_values=["00000", "00000 "],
#         )
#         display(df.head())
#     except Exception as e:
#         print(f"Error reading {file}: {e}")

### Main Content

* **Action Types**: lookup table of application actions (e.g. original application, review, etc...);
* **Submissions**: details on the applications, matching IDs to status, submission types and dates;
* **Products**: describes the drugs, with names and active ingredients;
* **Applications**: contains types of applications and the company which submitted it;
* **Marketing Status**: table and lookup for the market status of the drugs (e.g. over the counter, discontinued, etc...).

In [ ]:
# Load all products
products_df = pl.read_csv(
    os.path.join(FDA_DATA_PATH, "Products.txt"),
    separator="\t",
    ignore_errors=True,
    null_values=["00000", "00000 "],
)
products_df = (
    products_df
    .with_columns([
        pl.col("DrugName").str.strip_chars(
        ).str.to_lowercase().alias("drug_name"),
        pl.col("ActiveIngredient")
        .str.replace_all(" \\|\\| ", ";")
            .str.replace_all("\\|", ";")
            .str.replace_all(",", ";")
        .str.split(";")
        .alias("ActiveIngredient"),
    ])
    .explode("ActiveIngredient")
    .with_columns([
        pl.col("ActiveIngredient").str.strip_chars(
        ).str.to_lowercase().alias("active_ingredient")
    ])
    .filter(pl.col("active_ingredient").is_not_null() & (pl.col("active_ingredient") != ""))
)
print("Shape of products_df:", products_df.shape)
products_df = products_df.unique(
    subset=['drug_name', 'active_ingredient', 'ApplNo'])
print("Shape of products_df after removing duplicates:", products_df.shape)

Shape of products_df: (59207, 10)
Shape of products_df after removing duplicates: (35479, 10)


### Get approval dates

In [ ]:
# Load submissions to get approval dates
submissions_df = pl.read_csv(
    os.path.join(FDA_DATA_PATH, "Submissions.txt"),
    separator="\t",
    encoding="latin-1",
    ignore_errors=True,
)

# Get the earliest approval date per application
# Prioritize AP (full approval) over TA (tentative approval)
approval_dates = (
    submissions_df
    .filter(pl.col("SubmissionStatus").is_in(["AP", "TA"]))
    .with_columns([
        pl.col("SubmissionStatusDate").str.to_datetime(
            "%Y-%m-%d %H:%M:%S", strict=False).dt.date().alias("approval_date"),
        pl.when(pl.col("SubmissionStatus") == "AP").then(
            1).otherwise(2).alias("priority"),
    ])
    .sort(["ApplNo", "priority", "approval_date"])
    .group_by("ApplNo")
    .agg([
        pl.col("approval_date").first().alias("first_approval_date"),
        pl.col("SubmissionStatus").first().alias("approval_status"),
    ])
)

products_with_dates = products_df.join(
    approval_dates,
    on="ApplNo",
    how="left"
)

products_with_dates = products_with_dates.rename(
    {"ApplNo": "application_number"})

print(
    f"Products with approval dates: {products_with_dates.filter(pl.col('first_approval_date').is_not_null()).height} / {products_with_dates.height}")
products_with_dates.select(["drug_name", "active_ingredient",
                           "application_number", "first_approval_date", "approval_status"]).head(10)

Products with approval dates: 32262 / 35479


drug_name,active_ingredient,application_number,first_approval_date,approval_status
str,str,i64,date,str
"""niacin""","""niacin""",90860,2014-03-20,"""AP"""
"""aprepitant""","""aprepitant""",211835,2020-10-21,"""AP"""
"""nystex""","""nystatin""",62519,1984-07-06,"""AP"""
"""penicillamine""","""penicillamine""",211867,2020-08-04,"""AP"""
"""diazepam""","""diazepam""",70312,1985-12-16,"""AP"""
"""ondansetron hydrochloride""","""ondansetron hydrochloride""",77430,2007-06-27,"""AP"""
"""nicotine polacrilex""","""nicotine polacrilex""",90821,2009-07-10,"""AP"""
"""propranolol hydrochloride""","""propranolol hydrochloride""",70127,1985-07-30,"""AP"""
"""propoxyphene hydrochloride""","""propoxyphene hydrochloride""",84551,null,null


In [37]:
# Get Application Type (Biologics vs Small Molecules)
applications_df = pl.read_csv(
    os.path.join(FDA_DATA_PATH, "Applications.txt"),
    separator="\t",
    ignore_errors=True,
)

# BLA = Biologic License Application (biologics - novel)
# NDA = New Drug Application (small molecules - novel)
# ANDA = Abbreviated New Drug Application (generics - not novel)
print(applications_df['ApplType'].value_counts())

products_with_type = products_df.join(
    applications_df.select(["ApplNo", "ApplType"]),
    on="ApplNo",
    how="left"
).filter(
    pl.col("ApplType").is_not_null()  # Exclude incomplete entries
).with_columns(
    (pl.col("ApplType") == "BLA").alias("is_biologic"),
    (pl.col("ApplType").is_in(["NDA", "BLA"])).alias("is_novel"),
)

print(f"Products with type: {products_with_type.height}")
print(f"  - Novel (NDA+BLA): {products_with_type.filter(pl.col('is_novel')).height}")

shape: (3, 2)
┌──────────┬───────┐
│ ApplType ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ ANDA     ┆ 22416 │
│ NDA      ┆ 5809  │
│ BLA      ┆ 452   │
└──────────┴───────┘
Products with type: 35470
  - Novel (NDA+BLA): 9967


## Get targets

In [ ]:
# Access ChEMBL to get details on active ingredients and their targets
# Priority: 1. MOA from ChEMBL, 2. Open Targets, 3. Bioactivity
# Excludes CYP3A4 (CHEMBL340) - drug metabolism enzyme, not a real target

molecule_api = chembl_client.molecule
activity_api = chembl_client.activity
target_api = chembl_client.target
mechanism_api = chembl_client.mechanism

OPEN_TARGETS_URL = "https://api.platform.opentargets.org/api/v4/graphql"
CYP3A4_CHEMBL_ID = "CHEMBL340"
CYP3A4_UNIPROT_ID = "P08684"

TARGET_INFO_SCHEMA = {
    "target_chembl_id": pl.Utf8,
    "pref_name": pl.Utf8,
    "target_type": pl.Utf8,
    "component_name": pl.Utf8,
    "component_type": pl.Utf8,
    "uniprot_id": pl.Utf8,
}
TARGET_SCHEMA = {
    "mechanism_of_action": pl.Utf8,
    "action_type": pl.Utf8,
    "target_source": pl.Utf8,
    **TARGET_INFO_SCHEMA,
}

ACTIVITY_SCHEMA = {
    "molecule_chembl_id": pl.Utf8,
    "target_chembl_id": pl.Utf8,
    "mechanism_of_action": pl.Utf8,
    "action_type": pl.Utf8,
    "target_source": pl.Utf8,
}

FINAL_SCHEMA = {
    "molecule_chembl_id": pl.Utf8,
    "active_ingredient": pl.Utf8,
    "canonical_smiles": pl.Utf8,
    "standard_inchi_key": pl.Utf8,
    "max_phase": pl.Utf8,
    **TARGET_SCHEMA,
    **ACTIVITY_SCHEMA,
}

COMPONENT_RENAME_MAP = {
    "component_description": "component_name",
    "component_type": "component_type",
    "accession": "uniprot_id",
}


def normalize_drug_name(name: str) -> list[str]:
    """Generate name variants by removing common salt suffixes.

    Args:
        name: The name of the drug to normalize.

    Returns:
        A list of normalized drug names.
    """
    variants = [name]

    # Strip biosimilar suffix (4 lowercase letters after hyphen at end)
    biosimilar_match = re.match(r'^(.+)-[a-z]{4}$', name)
    if biosimilar_match:
        variants.append(biosimilar_match.group(1))

    # For ADCs like "ado-trastuzumab emtansine", try just the antibody name
    adc_match = re.match(r'^(?:ado-|fam-|bel-|pola-)?([\w]+)\s+\w+$', name)
    if adc_match:
        variants.append(adc_match.group(1))

    # Strip salt suffixes
    salt_patterns = [
        r'\s+(hydrochloride|hcl|sodium|potassium|sulfate|acetate|citrate|phosphate|maleate|fumarate|tartrate|mesylate|besylate)$',
        r'\s+\w+ate$',
    ]
    for pattern in salt_patterns:
        cleaned = re.sub(pattern, '', name, flags=re.IGNORECASE)
        if cleaned != name:
            variants.append(cleaned)
    return variants


def query_open_targets(chembl_id: str) -> list[dict] | None:
    """Query Open Targets for drug-target relationships with full target info.

    Args:
        chembl_id: ChEMBL molecule ID.

    Returns:
        List of target dicts with UniProt IDs and action types, or None if not found.
    """
    query = """
    query DrugTargets($chemblId: String!) {
        drug(chemblId: $chemblId) {
            mechanismsOfAction {
                rows {
                    actionType
                    mechanismOfAction
                    targets {
                        id
                        approvedSymbol
                        approvedName
                        proteinIds {
                            id
                            source
                        }
                    }
                }
            }
        }
    }
    """
    try:
        response = requests.post(
            OPEN_TARGETS_URL,
            json={"query": query, "variables": {"chemblId": chembl_id}},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        rows = data.get("data", {}).get("drug", {}).get(
            "mechanismsOfAction", {}).get("rows", [])
        if not rows:
            return None

        results = []
        for row in rows:
            action_type = row.get("actionType")
            moa = row.get("mechanismOfAction")
            for target in row.get("targets", []):
                uniprot_id = None
                for pid in target.get("proteinIds", []) or []:
                    if pid.get("source") == "uniprot_swissprot":
                        uniprot_id = pid.get("id")
                        break
                results.append({
                    "ensembl_id": target.get("id"),
                    "gene_symbol": target.get("approvedSymbol"),
                    "gene_name": target.get("approvedName"),
                    "uniprot_id": uniprot_id,
                    "action_type": action_type,
                    "mechanism_of_action": moa,
                })
        return results if results else None
    except Exception as e:
        vprint(f"Open Targets query failed: {e}")
        return None


def get_chembl_target_by_uniprot(uniprot_id: str) -> dict | None:
    """Look up ChEMBL target info by UniProt ID.

    Args:
        uniprot_id: UniProt accession.

    Returns:
        Dict with target_chembl_id and target_type, or None if not found.
    """
    if not uniprot_id:
        return None
    try:
        targets = target_api.filter(
            target_components__accession=uniprot_id
        ).only(["target_chembl_id", "target_type"])
        if targets:
            return {"target_chembl_id": targets[0]["target_chembl_id"], "target_type": targets[0]["target_type"]}
    except Exception:
        pass
    return None


def filter_cyp3a4(df: pl.DataFrame) -> pl.DataFrame:
    """Remove CYP3A4 entries from target dataframe (by ChEMBL ID or UniProt ID)."""
    if df.is_empty():
        return df
    filters = pl.lit(True)
    if "target_chembl_id" in df.columns:
        filters = filters & ((pl.col("target_chembl_id") != CYP3A4_CHEMBL_ID) | pl.col(
            "target_chembl_id").is_null())
    if "uniprot_id" in df.columns:
        filters = filters & (
            (pl.col("uniprot_id") != CYP3A4_UNIPROT_ID) | pl.col("uniprot_id").is_null())
    return df.filter(filters)


def query_drug(drug_name: str) -> pl.DataFrame | None:
    """Queries targets with priority: MOA > Open Targets > Bioactivity (excluding CYP3A4).

    Args:
        drug_name: The name of the drug to query.

    Returns:
        A Polars DataFrame with drug details and targets, or None if not found in ChEMBL.
        Drugs without structures (e.g., biologics) are included with NULL structural fields.
    """
    vprint(f"Processing drug: {drug_name}")

    # Search for the molecule
    molecules = None
    for variant in normalize_drug_name(drug_name):
        molecules = molecule_api.search(variant).only([
            "molecule_chembl_id", "pref_name", "molecule_structures", "max_phase", "standard_inchi_key",
        ])
        if molecules:
            break

    if not molecules:
        vprint(f"No molecule found for {drug_name}.")
        return None

    mol_data = molecules[0]
    
    # Handle drugs without structures (e.g., biologics) - keep them with NULL structural fields
    has_structure = mol_data.get("molecule_structures") is not None
    if not has_structure:
        vprint(f"No structure for {drug_name} (likely a biologic). Keeping with NULL structural fields.")
    
    mol_df = pl.DataFrame({
        "active_ingredient": drug_name,
        "molecule_chembl_id": mol_data["molecule_chembl_id"],
        "max_phase": mol_data["max_phase"],
        "canonical_smiles": mol_data["molecule_structures"]["canonical_smiles"] if has_structure else None,
        "standard_inchi_key": mol_data["molecule_structures"]["standard_inchi_key"] if has_structure else None,
    })
    chembl_id = mol_data["molecule_chembl_id"]

    mechanisms_df = None

    # Priority 1: MOA from ChEMBL
    mechanisms = mechanism_api.filter(molecule_chembl_id=chembl_id).only([
        "molecule_chembl_id", "target_chembl_id", "mechanism_of_action", "action_type",
    ])

    if mechanisms:
        vprint(f"Found MOA for {drug_name}")
        mechanisms_df = pl.DataFrame(
            [dict(m) for m in mechanisms], schema_overrides=ACTIVITY_SCHEMA
        ).with_columns(pl.lit("chembl_moa").alias("target_source"))
        mechanisms_df = filter_cyp3a4(mechanisms_df)

    # Priority 2: Open Targets (if no MOA)
    if mechanisms_df is None or mechanisms_df.is_empty():
        ot_targets = query_open_targets(chembl_id)
        if ot_targets:
            vprint(f"Found Open Targets data for {drug_name}")
            ot_rows = []
            for t in ot_targets:
                # Look up ChEMBL target info via UniProt
                chembl_info = get_chembl_target_by_uniprot(t.get("uniprot_id"))
                ot_rows.append({
                    "molecule_chembl_id": chembl_id,
                    "target_chembl_id": chembl_info["target_chembl_id"] if chembl_info else None,
                    "mechanism_of_action": t.get("mechanism_of_action") or f"Target: {t.get('gene_symbol', 'Unknown')}",
                    "action_type": t.get("action_type") or "UNKNOWN",
                    "target_source": "open_targets",
                    "pref_name": t.get("gene_name"),
                    "component_name": t.get("gene_name"),
                    "uniprot_id": t.get("uniprot_id"),
                    "target_type": chembl_info["target_type"] if chembl_info else None,
                })
            if ot_rows:
                ot_schema = {**ACTIVITY_SCHEMA, "pref_name": pl.Utf8,
                             "component_name": pl.Utf8, "uniprot_id": pl.Utf8, "target_type": pl.Utf8}
                mechanisms_df = pl.DataFrame(ot_rows, schema=ot_schema)
                mechanisms_df = filter_cyp3a4(mechanisms_df)

    # Priority 3: Bioactivity (if no MOA and no Open Targets)
    if mechanisms_df is None or mechanisms_df.is_empty():
        vprint(f"Falling back to bioactivity for {drug_name}")
        activities = (
            activity_api.filter(
                molecule_chembl_id=chembl_id,
                target_organism="Homo sapiens",
                assay_type__in=["B"],
                pchembl_value__isnull=False,
                standard_type__in=["IC50", "Ki", "EC50"]
            )
            .order_by("-pchembl_value")
            .only(["molecule_chembl_id", "target_chembl_id", "standard_type"])
        )[:5]

        if activities:
            mechanisms_df = pl.DataFrame([{
                "molecule_chembl_id": a.get("molecule_chembl_id"),
                "target_chembl_id": a.get("target_chembl_id"),
                "mechanism_of_action": f"Binding ({a.get('standard_type', 'Unknown')})",
                "action_type": "BINDER",
                "target_source": "chembl_bioactivity",
            } for a in activities], schema=ACTIVITY_SCHEMA)
            mechanisms_df = filter_cyp3a4(mechanisms_df)

    # No targets found
    if mechanisms_df is None or mechanisms_df.is_empty():
        vprint(f"No target data found for {drug_name}")

        result = mol_df
        for col in FINAL_SCHEMA.keys():
            if col not in result.columns:
                result = result.with_columns(
                    pl.lit(None).cast(pl.Utf8).alias(col))
        return result.select(sorted(FINAL_SCHEMA.keys()))

    # Get target details (only needed if we have ChEMBL target IDs)
    valid_target_ids = [
        tid for tid in mechanisms_df["target_chembl_id"].unique().to_list()
        if tid is not None
    ]

    if valid_target_ids:
        target_results = target_api.filter(target_chembl_id__in=valid_target_ids).only([
            "target_chembl_id", "pref_name", "target_components", "target_type"
        ])
        targets = pl.DataFrame([dict(t) for t in target_results])

        if not targets.is_empty() and "target_components" in targets.columns:
            targets = targets.explode("target_components")
            field_names = [f.name for f in targets.schema["target_components"].fields]

            cols_to_rename = [
                pl.col("target_components").struct.field(old_name).alias(new_name)
                if old_name in field_names else
                pl.lit(None).cast(pl.Utf8).alias(new_name)
                for old_name, new_name in COMPONENT_RENAME_MAP.items()
            ]
            targets = targets.with_columns(cols_to_rename)

            targets = targets.drop("target_components")

        for col in TARGET_INFO_SCHEMA.keys():
            if col not in targets.columns:
                targets = targets.with_columns(
                    pl.lit(None).cast(pl.Utf8).alias(col))
            else:
                targets = targets.with_columns(
                    pl.col(col).cast(pl.Utf8)
                )

        merged = mechanisms_df.join(targets, on="target_chembl_id", how="left")
    else:
        merged = mechanisms_df

    # Add molecule data
    merged = merged.join(mol_df, on="molecule_chembl_id", how="left")
    for col in FINAL_SCHEMA.keys():
        if col not in merged.columns:
            merged = merged.with_columns(pl.lit(None).alias(col))
        merged = merged.with_columns(pl.col(col).cast(pl.Utf8))

    return merged.select(sorted(FINAL_SCHEMA.keys()))


def query_multiple_drugs(drug_names: list[str]) -> pl.DataFrame:
    """
    Processes a list of drug names, applying a rate limit between calls
    to comply with API usage policies.

    Args:
        drug_names: List of drug names to query.

    Returns:
        Combined DataFrame with results from all drugs.
    """
    all_results = []
    successes = 0

    for drug_name in tqdm(drug_names, desc="Querying drugs", unit="drug", leave=False):
        try:
            result = query_drug(drug_name)
            if result is not None:
                successes += 1
                all_results.append(result)
            else:
                raise Exception(f"No result found for {drug_name}")
            sleep(API_DELAY_SECONDS)
        except Exception as e:
            vprint(f"ERROR processing {drug_name}: {e}")
            sleep(5)
            continue

    print(f"Success rate: {successes / len(drug_names):.2%}")

    if not all_results:
        print("No results were successfully fetched.")
        return pl.DataFrame()

    return pl.concat(all_results)

In [38]:
# Now download the data from ChEMBL

# test_drug_names = ["imatinib", "aspirin", "metformin", "lisinopril", "atorvastatin"]
# drug_table = query_multiple_drugs(test_drug_names)

os.environ['VERBOSE'] = 'False'

batch_size = 500
drug_names = sorted(products_df['active_ingredient'].unique().to_list())
print("Number of unique active ingredients:", len(drug_names))

batched_tables = []
for batch in tqdm(range(0, len(drug_names), batch_size),
                  total=len(drug_names)//batch_size,
                  desc="Batching drugs", unit="batch", leave=False):
    # drug_table = query_multiple_drugs(drug_names[batch:batch+batch_size])
    drug_table = pl.read_parquet(os.path.join(DATA_PATH, f"drug_table_{batch/batch_size}.pq"))
    batched_tables.append(drug_table)
    # drug_table.write_parquet(os.path.join(DATA_PATH, f"drug_table_{batch/batch_size}.pq"))

drug_table = pl.concat(batched_tables)

Number of unique active ingredients: 2820


Batching drugs:   0%|          | 0/5 [00:00<?, ?batch/s]

In [ ]:
# Join approval dates AND biologic flag to drug_table
# Use the EARLIEST approval date for each active ingredient (fixes combination drug dating issue)
# Prioritize AP (full approval) over TA (tentative approval)

# Get the earliest approval date per active ingredient, preferring AP over TA
ingredient_first_approval = (
    products_with_dates
    .filter(pl.col("first_approval_date").is_not_null())
    .with_columns(
        pl.when(pl.col("approval_status") == "AP").then(1).otherwise(2).alias("priority")
    )
    .sort(["active_ingredient", "priority", "first_approval_date"])
    .group_by("active_ingredient")
    .agg(
        pl.col("first_approval_date").first().alias("first_approval_date"),
        pl.col("approval_status").first().alias("approval_status"),
    )
)

# Get application type flags per active ingredient
# is_biologic: True if ANY application is BLA
# is_novel: True if ANY application is NDA or BLA (not generic)
appl_type_flags = (
    products_df.join(
        applications_df.select(["ApplNo", "ApplType"]),
        on="ApplNo",
        how="left"
    )
    .filter(pl.col("ApplType").is_not_null())
    .with_columns([
        (pl.col("ApplType") == "BLA").alias("is_biologic"),
        (pl.col("ApplType").is_in(["NDA", "BLA"])).alias("is_novel"),
    ])
    .group_by("active_ingredient")
    .agg([
        pl.col("is_biologic").max(),
        pl.col("is_novel").max(),  # True if ANY novel application exists
    ])
)

# Join both to drug_table
drug_table = (
    drug_table
    .join(ingredient_first_approval, on="active_ingredient", how="left")
    .join(appl_type_flags, on="active_ingredient", how="left")
)

print(f"Drugs with approval dates: {drug_table.filter(pl.col('first_approval_date').is_not_null()).height} / {drug_table.height}")
print(f"Biologics: {drug_table.filter(pl.col('is_biologic') == True).height}")
print(f"Novel (NDA+BLA): {drug_table.filter(pl.col('is_novel') == True).height}")


Drugs with approval dates: 11707 / 12152
Biologics: 62
Novel (NDA+BLA): 9223


In [40]:
# Save the final table
drug_table.write_parquet(os.path.join(DATA_PATH, "drug_table_251218.pq"))

## Get target annotations

In [15]:
# Get unique targets from drug_table
drug_table = pl.read_parquet(os.path.join(DATA_PATH, "drug_table_251218.pq"))

unique_targets = (
    drug_table
    .select(["target_chembl_id", "uniprot_id"])
    .unique()
    .filter(pl.col("target_chembl_id").is_not_null() | pl.col("uniprot_id").is_not_null())
)
print(f"Found {len(unique_targets)} unique targets")
unique_targets.head()

Found 1485 unique targets


target_chembl_id,uniprot_id
str,str
"""CHEMBL2363965""","""P9WHB7"""
"""CHEMBL2311225""","""P72524"""
"""CHEMBL2364103""","""Q9A1W4"""
"""CHEMBL2363135""","""P68679"""
"""CHEMBL2364701""","""Q16186"""


In [16]:
# Get annotations from ChEMBL
PROTEIN_CLASS_SCHEMA = {
    "target_chembl_id": pl.Utf8,
    "target_type": pl.Utf8,      # SINGLE PROTEIN, PROTEIN COMPLEX, etc.
    "panther_family": pl.Utf8,   # High-level protein family (e.g., G-PROTEIN COUPLED RECEPTOR)
    "interpro_domain": pl.Utf8,  # InterPro domain (e.g., GPCR_Rhodpsn)
    "pfam_family": pl.Utf8,      # Pfam family (e.g., 7tm_1)
}


def fetch_protein_classes(target_chembl_ids: list[str], chunk_size: int = 50) -> pl.DataFrame:
    """Fetches protein classification from ChEMBL target xrefs.
    
    Extracts meaningful classification from:
    - target_type: broad category (SINGLE PROTEIN, PROTEIN COMPLEX, etc.)
    - PANTHER: high-level protein family
    - InterPro: protein domains
    - Pfam: protein families
    
    Args:
        target_chembl_ids: List of ChEMBL target IDs.
        chunk_size: Number of IDs to query per API call.
        
    Returns:
        DataFrame with target classification info.
    """
    all_classes = []
    valid_ids = list(set([tid for tid in target_chembl_ids if tid]))
    
    for i in tqdm(range(0, len(valid_ids), chunk_size), desc="Fetching protein classes"):
        chunk = valid_ids[i : i + chunk_size]
        
        try:
            targets = target_api.filter(target_chembl_id__in=chunk)

            for target in targets:
                tid = target["target_chembl_id"]
                target_type = target.get("target_type")
                
                # Extract classifications from xrefs
                panther_families = []
                interpro_domains = []
                pfam_families = []
                
                for component in target.get("target_components", []):
                    for xref in component.get("target_component_xrefs", []):
                        db = xref.get("xref_src_db")
                        name = xref.get("xref_name")
                        
                        if db == "PANTHER" and name:
                            panther_families.append(name)
                        elif db == "InterPro" and name:
                            interpro_domains.append(name)
                        elif db == "Pfam" and name:
                            pfam_families.append(name)
                
                # Take first of each (or None)
                all_classes.append({
                    "target_chembl_id": tid,
                    "target_type": target_type,
                    "panther_family": panther_families[0] if panther_families else None,
                    "interpro_domain": interpro_domains[0] if interpro_domains else None,
                    "pfam_family": pfam_families[0] if pfam_families else None,
                })
                    
        except Exception as e:
            print(f"Error fetching chunk starting {chunk[0]}: {e}")
            continue

    if not all_classes:
        return pl.DataFrame(schema=PROTEIN_CLASS_SCHEMA)
    
    return pl.DataFrame(all_classes, schema=PROTEIN_CLASS_SCHEMA).unique()

# Fetch protein classes for all unique ChEMBL target IDs
target_chembl_ids = unique_targets["target_chembl_id"].unique().to_list()
protein_classes_df = fetch_protein_classes(target_chembl_ids)
print(f"Found {len(protein_classes_df)} protein classifications")
print(protein_classes_df.head())

Fetching protein classes:   0%|          | 0/12 [00:00<?, ?it/s]

Found 573 protein classifications
shape: (5, 5)
┌──────────────────┬────────────────┬────────────────────────┬───────────────────────┬─────────────┐
│ target_chembl_id ┆ target_type    ┆ panther_family         ┆ interpro_domain       ┆ pfam_family │
│ ---              ┆ ---            ┆ ---                    ┆ ---                   ┆ ---         │
│ str              ┆ str            ┆ str                    ┆ str                   ┆ str         │
╞══════════════════╪════════════════╪════════════════════════╪═══════════════════════╪═════════════╡
│ CHEMBL1955       ┆ SINGLE PROTEIN ┆ TYROSINE-PROTEIN       ┆ Ig-like_dom           ┆ I-set       │
│                  ┆                ┆ KINASE RECEPT…         ┆                       ┆             │
│ CHEMBL222        ┆ SINGLE PROTEIN ┆ SODIUM-DEPENDENT       ┆ Na/ntran_symport      ┆ SNF         │
│                  ┆                ┆ NORADRENALINE…         ┆                       ┆             │
│ CHEMBL2007625    ┆ SINGLE PROTEIN ┆ ISOCI

In [17]:
# Fetch IUPHAR/Guide to Pharmacology target classification
# Data available at: https://www.guidetopharmacology.org/download.jsp

IUPHAR_ADDRESS = 'https://www.guidetopharmacology.org/DATA/targets_and_families.csv'

def fetch_iuphar_classification() -> pl.DataFrame:
    """Fetches IUPHAR target classification from Guide to Pharmacology.
    
    Returns:
        DataFrame with uniprot_id and IUPHAR family hierarchy.
    """
    
    iuphar_df = pl.read_csv(IUPHAR_ADDRESS,  skip_rows=1).select([
        pl.col("Human SwissProt").alias("uniprot_id"),
        pl.col("Type").alias("iuphar_type"),          
        pl.col("Family name").alias("iuphar_family"),   
        pl.col("Target name").alias("iuphar_target"), 
    ]).filter(pl.col("uniprot_id").is_not_null()).unique()
    
    return iuphar_df

# Fetch IUPHAR data
iuphar_df = fetch_iuphar_classification()
print(f"Found {len(iuphar_df)} IUPHAR target annotations")
print(iuphar_df.head(10))

Found 3352 IUPHAR target annotations
shape: (10, 4)
┌────────────┬────────────────────┬─────────────────────────────────┬───────────────────────────┐
│ uniprot_id ┆ iuphar_type        ┆ iuphar_family                   ┆ iuphar_target             │
│ ---        ┆ ---                ┆ ---                             ┆ ---                       │
│ str        ┆ str                ┆ str                             ┆ str                       │
╞════════════╪════════════════════╪═════════════════════════════════╪═══════════════════════════╡
│ P31321     ┆ enzyme             ┆ Protein kinase A (PKA) family   ┆ protein kinase,           │
│            ┆                    ┆                                 ┆ cAMP-dependent…           │
│ Q8IWK6     ┆ gpcr               ┆ Adhesion Class GPCRs            ┆ ADGRA3                    │
│ P14616     ┆ catalytic_receptor ┆ Type II RTKs: Insulin receptor… ┆ Insulin receptor-related  │
│            ┆                    ┆                               

In [18]:

GO_SCHEMA = {
    "uniprot_id": pl.Utf8,
    "go_id": pl.Utf8,
    "go_term": pl.Utf8,
    "go_aspect": pl.Utf8,  # P=biological process, F=molecular function, C=cellular component
}



def fetch_go_terms(uniprot_ids: list[str], batch_size: int = 100) -> pl.DataFrame:
    """Fetches GO terms from UniProt for given protein IDs.
    
    Args:
        uniprot_ids: List of UniProt accession IDs.
        batch_size: Number of IDs to query at once.
        
    Returns:
        DataFrame with uniprot_id, go_id, go_term, and go_aspect.
    """
    base_url = "https://rest.uniprot.org/uniprotkb/stream"
    
    all_go_terms = []
    valid_ids = [uid for uid in uniprot_ids if uid is not None]
    
    for i in tqdm(range(0, len(valid_ids), batch_size), desc="Fetching GO terms", leave=False):
        batch = valid_ids[i:i + batch_size]
        query = " OR ".join([f"accession:{uid}" for uid in batch])
        
        params = {
            "query": query,
            "fields": "accession,go_p,go_f,go_c",
            "format": "json",
        }
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            for entry in data.get("results", []):
                accession = entry.get("primaryAccession")
                
                # Process GO terms from different aspects
                go_mappings = {
                    "P": entry.get("goTerms", {}).get("biologicalProcess", []),
                    "F": entry.get("goTerms", {}).get("molecularFunction", []),
                    "C": entry.get("goTerms", {}).get("cellularComponent", []),
                }
                
                # UniProt API returns GO terms in uniProtKBCrossReferences
                for ref in entry.get("uniProtKBCrossReferences", []):
                    if ref.get("database") == "GO":
                        go_id = ref.get("id")
                        props = {p.get("key"): p.get("value") for p in ref.get("properties", [])}
                        go_term = props.get("GoTerm", "")
                        
                        # Extract aspect from GoTerm prefix (C:, F:, P:)
                        aspect = ""
                        if go_term.startswith("C:"):
                            aspect = "C"
                            go_term = go_term[2:]
                        elif go_term.startswith("F:"):
                            aspect = "F"
                            go_term = go_term[2:]
                        elif go_term.startswith("P:"):
                            aspect = "P"
                            go_term = go_term[2:]
                        
                        all_go_terms.append({
                            "uniprot_id": accession,
                            "go_id": go_id,
                            "go_term": go_term,
                            "go_aspect": aspect,
                        })
            
            sleep(API_DELAY_SECONDS)
            
        except Exception as e:
            vprint(f"Error fetching GO terms for batch starting at {i}: {e}")
            continue
    
    if not all_go_terms:
        return pl.DataFrame(schema=GO_SCHEMA)
    
    return pl.DataFrame(all_go_terms, schema=GO_SCHEMA).unique()

# Fetch GO terms for all unique UniProt IDs
uniprot_ids = unique_targets["uniprot_id"].unique().to_list()
go_terms_df = fetch_go_terms(uniprot_ids)
print(f"Found {len(go_terms_df)} GO term annotations")
go_terms_df.head()

Fetching GO terms:   0%|          | 0/13 [00:00<?, ?it/s]

Found 20001 GO term annotations


uniprot_id,go_id,go_term,go_aspect
str,str,str,str
"""Q14721""","""GO:0048471""","""perinuclear region of cytoplas…","""C"""
"""P11388""","""GO:0000793""","""condensed chromosome""","""C"""
"""Q07864""","""GO:0006287""","""base-excision repair, gap-fill…","""P"""
"""P69905""","""GO:0015670""","""carbon dioxide transport""","""P"""
"""P31751""","""GO:0090314""","""positive regulation of protein…","""P"""


In [36]:
# Save annotation tables
protein_classes_df.write_parquet(os.path.join(DATA_PATH, "protein_classes.pq"))
iuphar_df.write_parquet(os.path.join(DATA_PATH, "iuphar.pq"))
go_terms_df.write_parquet(os.path.join(DATA_PATH, "go_terms.pq"))
print("Saved annotations to parquet files")

Saved annotations to parquet files


In [41]:
protein_classes_df = pl.read_parquet(os.path.join(DATA_PATH, "protein_classes.pq"))
iuphar_df = pl.read_parquet(os.path.join(DATA_PATH, "iuphar.pq"))
go_terms_df = pl.read_parquet(os.path.join(DATA_PATH, "go_terms.pq"))

# Join annotations WITHOUT GO terms (keeps row count manageable)
# GO terms can be joined separately when needed for specific analyses
drug_table_annotated = (
    drug_table
    .join(protein_classes_df, on="target_chembl_id", how="left", suffix="_pc")
    .join(iuphar_df, on="uniprot_id", how="left")
    #.join(go_terms_df, on="uniprot_id", how="left")
    .with_columns(
        pl.col("first_approval_date").dt.year().alias("approval_year")
    )
)
drug_table_annotated.write_parquet(os.path.join(DATA_PATH, "drug_table_annotated.pq"))
print(f"Annotated drug table shape: {drug_table_annotated.shape}")
print(f"Date range: {drug_table_annotated['approval_year'].min()} - {drug_table_annotated['approval_year'].max()}")
print(f"Biologics in table: {drug_table_annotated.filter(pl.col('is_biologic') == True).height}")
drug_table_annotated.head()


Annotated drug table shape: (12250, 26)
Date range: 1939 - 2025
Biologics in table: 62


action_type,active_ingredient,canonical_smiles,component_name,component_type,max_phase,mechanism_of_action,molecule_chembl_id,pref_name,standard_inchi_key,target_chembl_id,target_source,target_type,uniprot_id,first_approval_date,approval_status,is_biologic,is_novel,target_type_pc,panther_family,interpro_domain,pfam_family,iuphar_type,iuphar_family,iuphar_target,approval_year
str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,str,bool,bool,str,str,str,str,str,str,str,i32
"""INHIBITOR""","""abacavir""","""Nc1nc(NC2CC2)c2ncn([C@H]3C=C[C…","""Reverse transcriptase/RNaseH""","""PROTEIN""","""4.0""","""Human immunodeficiency virus t…","""CHEMBL4303288""","""Human immunodeficiency virus t…","""WMHSRBZIJNQHKT-FFKFEZPRSA-N""","""CHEMBL247""","""chembl_moa""","""SINGLE PROTEIN""","""Q72547""",2006-07-26,"""TA""",false,true,"""SINGLE PROTEIN""","""ENDOGENOUS RETROVIRUS GROUP K …","""RT_dom""","""RNase_H""",null,null,null,2006
"""INHIBITOR""","""abacavir sulfate""","""Nc1nc(NC2CC2)c2ncn([C@H]3C=C[C…","""Reverse transcriptase/RNaseH""","""PROTEIN""","""4.0""","""Human immunodeficiency virus t…","""CHEMBL4303288""","""Human immunodeficiency virus t…","""WMHSRBZIJNQHKT-FFKFEZPRSA-N""","""CHEMBL247""","""chembl_moa""","""SINGLE PROTEIN""","""Q72547""",1998-12-17,"""AP""",false,true,"""SINGLE PROTEIN""","""ENDOGENOUS RETROVIRUS GROUP K …","""RT_dom""","""RNase_H""",null,null,null,1998
"""ANTAGONIST""","""abarelix""","""CC(=O)N[C@H](Cc1ccc2ccccc2c1)C…","""Gonadotropin-releasing hormone…","""PROTEIN""","""4.0""","""Gonadotropin-releasing hormone…","""CHEMBL1252""","""Gonadotropin-releasing hormone…","""AIWRTTMUVOZGPW-HSPKUQOVSA-N""","""CHEMBL1855""","""chembl_moa""","""SINGLE PROTEIN""","""P30968""",2003-11-25,"""AP""",false,true,"""SINGLE PROTEIN""","""GONADOTROPIN-RELEASING HORMONE…","""GPCR_Rhodpsn""","""7tm_1""","""gpcr""","""Gonadotrophin-releasing hormon…","""GnRH<sub>1</sub> receptor""",2003
"""INHIBITOR""","""abemaciclib""","""CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4c…","""Cyclin-dependent kinase 4""","""PROTEIN""","""4.0""","""Cyclin-dependent kinase 4 inhi…","""CHEMBL3301610""","""Cyclin-dependent kinase 4""","""UZWDCWONPYILKI-UHFFFAOYSA-N""","""CHEMBL331""","""chembl_moa""","""SINGLE PROTEIN""","""P11802""",2017-09-28,"""AP""",false,true,"""SINGLE PROTEIN""","""CELL DIVISION PROTEIN KINASE""","""CDK""","""Pkinase""","""enzyme""","""CDK4 subfamily""","""cyclin dependent kinase 4""",2017
"""INHIBITOR""","""abemaciclib""","""CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4c…","""Cyclin-dependent kinase 6""","""PROTEIN""","""4.0""","""Cyclin-dependent kinase 6 inhi…","""CHEMBL3301610""","""Cyclin-dependent kinase 6""","""UZWDCWONPYILKI-UHFFFAOYSA-N""","""CHEMBL2508""","""chembl_moa""","""SINGLE PROTEIN""","""Q00534""",2017-09-28,"""AP""",false,true,"""SINGLE PROTEIN""","""CELL DIVISION PROTEIN KINASE""","""CDK""","""Pkinase""","""enzyme""","""CDK4 subfamily""","""cyclin dependent kinase 6""",2017


## Structural Analysis

In [42]:
drug_table_annotated = pl.read_parquet(os.path.join(DATA_PATH, "drug_table_annotated.pq"))

compounds = drug_table_annotated['canonical_smiles'].unique().to_list()

def compute_structural_props(smiles: str) -> dict:
    """
    Computes structural properties for a given SMILES string.
    Returns None if the SMILES is invalid.
    """
    if not smiles:
        return None
    
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None
        
    try:
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        hbd = Descriptors.NumHDonors(mol)
        hba = Descriptors.NumHAcceptors(mol)
        
        # Lipinski Violations (Rule of 5)
        violations = 0
        if mw > 500: violations += 1
        if logp > 5: violations += 1
        if hbd > 5: violations += 1
        if hba > 10: violations += 1
        
        scaffold_mol = MurckoScaffold.GetScaffoldForMol(mol)
        scaffold_smiles = Chem.MolToSmiles(scaffold_mol)
        
        return {
            "canonical_smiles": smiles,
            "molecular_weight": mw,
            "logp": logp,
            "h_bond_donors": hbd,
            "h_bond_acceptors": hba,
            "lipinski_violations": violations,
            "scaffold_smiles": scaffold_smiles
        }
    except Exception:
        return None

prop_schema = pl.Struct({
    "smiles": pl.Utf8,
    "molecular_weight": pl.Float64,
    "logp": pl.Float64,
    "h_bond_donors": pl.Int64,
    "h_bond_acceptors": pl.Int64,
    "lipinski_violations": pl.Int64,
    "scaffold_smiles": pl.Utf8
})


# Compute structural properties
print("Calculating structural properties...")
compounds_annotated = []
for compound in tqdm(compounds, desc="Computing structural properties"):
    props = compute_structural_props(compound)
    if props:
        compounds_annotated.append(props)

compounds_annotated = pl.DataFrame(compounds_annotated)
print("Annotated compounds table shape:", compounds_annotated.shape)

drug_table_structures = drug_table_annotated.join(compounds_annotated, on="canonical_smiles", how="left")
print("Drug table annotated with structures shape:", drug_table_structures.shape)
drug_table_structures.head()

drug_table_structures.write_parquet(os.path.join(DATA_PATH, "drug_table_annotated_structures.pq"))

Calculating structural properties...


Computing structural properties:   0%|          | 0/1973 [00:00<?, ?it/s]

Annotated compounds table shape: (1973, 7)
Drug table annotated with structures shape: (12250, 32)


## FDA Summary Table (All Drugs Including Biologics)

The ChEMBL-based drug_table excludes many biologics because they don't have small molecule structures. 
This section creates a separate FDA summary table directly from FDA data for proportion/trend analyses.

In [43]:
# Create FDA summary table with ALL drugs (including biologics not in ChEMBL)
# This table is for proportion/trend analyses, not structural analysis

# Get unique active ingredients with their earliest approval and type flags
fda_summary = (
    products_with_dates
    .join(
        applications_df.select(["ApplNo", "ApplType"]).rename({"ApplNo": "application_number"}),
        on="application_number",
        how="left"
    )
    .filter(pl.col("ApplType").is_not_null())  # Exclude entries with null ApplType
    .with_columns([
        (pl.col("ApplType") == "BLA").alias("is_biologic"),
        (pl.col("ApplType").is_in(["NDA", "BLA"])).alias("is_novel"),  # Novel = NDA or BLA
        pl.col("first_approval_date").dt.year().alias("approval_year"),
    ])
    .filter(pl.col("first_approval_date").is_not_null())
    # Get earliest approval per active ingredient, preferring AP over TA
    .with_columns(
        pl.when(pl.col("approval_status") == "AP").then(1).otherwise(2).alias("priority")
    )
    .sort(["active_ingredient", "priority", "first_approval_date"])
    .group_by("active_ingredient")
    .agg([
        pl.col("first_approval_date").first(),
        pl.col("approval_status").first(),
        pl.col("approval_year").first(),
        pl.col("is_biologic").max(),  # True if ANY application is BLA
        pl.col("is_novel").max(),     # True if ANY application is novel (NDA/BLA)
        pl.col("drug_name").first(),
    ])
)

# Also check if drug is in ChEMBL (has structural data)
drug_table_structures = pl.read_parquet(os.path.join(DATA_PATH, "drug_table_annotated_structures.pq"))
chembl_ingredients = drug_table_structures['active_ingredient'].unique().to_list()
fda_summary = fda_summary.with_columns(
    pl.col("active_ingredient").is_in(chembl_ingredients).alias("in_chembl")
)

print(f"FDA Summary Table: {fda_summary.height} unique active ingredients")
print(f"  - Biologics (BLA): {fda_summary.filter(pl.col('is_biologic')).height}")
print(f"  - Novel (NDA+BLA): {fda_summary.filter(pl.col('is_novel')).height}")
print(f"  - In ChEMBL: {fda_summary.filter(pl.col('in_chembl')).height}")
print(f"  - Not in ChEMBL: {fda_summary.filter(~pl.col('in_chembl')).height}")

# Save FDA summary
fda_summary.write_parquet(os.path.join(DATA_PATH, "fda_summary.pq"))
print(f"\nSaved to {os.path.join(DATA_PATH, 'fda_summary.pq')}")
fda_summary.head(10)


FDA Summary Table: 2740 unique active ingredients
  - Biologics (BLA): 384
  - Novel (NDA+BLA): 2633
  - In ChEMBL: 2041
  - Not in ChEMBL: 699

Saved to /Users/federico.comitani/Work/inference/data/fda_summary.pq


active_ingredient,first_approval_date,approval_status,approval_year,is_biologic,is_novel,drug_name,in_chembl
str,date,str,i32,bool,bool,str,bool
"""abacavir""",2006-07-26,"""TA""",2006,false,true,"""lamivudine; zidovudine; abacav…",true
"""abacavir sulfate""",1998-12-17,"""AP""",1998,false,true,"""ziagen""",true
"""abaloparatide""",2017-04-28,"""AP""",2017,false,true,"""tymlos""",false
"""abametapir""",2020-07-24,"""AP""",2020,false,true,"""xeglyze""",false
"""abarelix""",2003-11-25,"""AP""",2003,false,true,"""plenaxis""",true
"""abatacept""",2005-12-23,"""AP""",2005,true,true,"""orencia""",false
"""abciximab""",1994-12-22,"""AP""",1994,true,true,"""reopro""",false
"""abemaciclib""",2017-09-28,"""AP""",2017,false,true,"""verzenio""",true
"""abiraterone acetate""",2011-04-28,"""AP""",2011,false,true,"""zytiga""",true
